# Spatial Solver Tutorial

`spatialsolver` is a pyne module that contains seven neutron transport equation solvers.
The neutron transport equation is a balance statement that conserves neutrons.

In [33]:
import pyne
import pyne.spatialsolver
import numpy as np

In [34]:
input_dict = {'Name': 'Jane', 'Age': 27}

The `spatialsolver` module takes in a dictionary that contains all of the input information required to run the solvers. There are many entries to allow a high degree of customization, not all of which are required.  To find which entries are required, see the spatial solver documentation in the python api.

In [35]:
input_dict['solver'] = "AHOTN"

The neutron transport equations can be solved using various methods, and the spatial solver supports three different families of solvers. These families, along with their corresponding keys, are described in the theory manual as follows:

- **AHOTN** - Arbitrarily Higher Order Transport Method: This method involves solving the neutron transport equations using a higher-order scheme, allowing for increased accuracy in the solution.

- **DGFEM** - Discontinuous Galerkin Finite Element Method: The DGFEM method employs a finite element approach with discontinuous basis functions to solve the transport equations. This method is particularly useful for handling complex geometries and capturing discontinuities in the solution.

- **SCTSTEP** - SCT Step Algorithm: This algorithm, similar to Duo's SCT algorithm, is implemented in three-dimensional Cartesian geometry. It provides a specific approach for solving the neutron transport equations in this geometric configuration.

By selecting the appropriate "solver" key, you can choose which family of solvers you would like to use for solving the neutron transport equations.

In [36]:
input_dict['solver_type'] = "LN" 

Among the families of solvers available, except for SCTSTEP, each family offers multiple choices for solving the neutron transport equation. Here are the specific options for each family:

AHOTN:

- LN - Arbitrarily Higher Order Transport Method of the Nodal Type, using the linear-nodal method.
- LL - Arbitrarily Higher Order Transport Method of the Nodal Type, using the linear-linear method.
- NEFD - Arbitrarily Higher Order Transport Method of the Nodal Type, utilizing the unknown nodal flux moments (NEFD algorithm).

DGFEM:

- LD - Discontinuous Galerkin Finite Element Method (DGFEM) with a linear discontinuous (LD) approximation for the angular flux.
- DENSE - Discontinuous Galerkin Finite Element Method (DGFEM) using dense Lagrange polynomials.
- LAGRANGE - Discontinuous Galerkin Finite Element Method (DGFEM) employing Lagrange polynomials.

SCTSTEP:
- SCT Step algorithm, similar to Duo's SCT algorithm, implemented in three-dimensional Cartesian geometry.

In [37]:
input_dict['spatial_order'] = 1

The Spatial expansion order is the expansion order of the spatial moment.  It is also known as lambda, and for all AHOTN solvers it must be 0, 1 or 2.

In [38]:
input_dict['angular_quadrature_order'] = 4

  The angular quadrature order is the number of angles to be used per octant.  
  For N sets of angles, there will be (N * (N + 2) / 8) ordinates per octant. 
  The quadrature order may only be an even number!

In [39]:
input_dict['angular_quadrature_type'] = 1

  The quadrature type is the type of quadrature scheme the code uses.  
  The possibilities are:
  
- TWOTRAN
- EQN
- Read-in

In [40]:
input_dict['nodes_xyz'] = [4,4,4]

The variable `nodes_xyz` represents the number of nodes in the x, y, and z directions. It should be stored as a 1D array with three entries in the following order:

- The first entry corresponds to the number of nodes in the x direction (integer).
- The second entry corresponds to the number of nodes in the y direction (integer).
- The third entry corresponds to the number of nodes in the z direction (integer).

In [41]:
input_dict['num_groups'] = 1

`num_groups` specifies the number of material groups you are using in the material id and cross section files found in later entries.

In [42]:
input_dict['num_materials'] = 1

`num_materials` is the number of different materials used in the mesh ('material_id').

In [43]:
input_dict['x_cells_widths'] = [0.25, 0.25, 0.25, 0.25]

In [44]:
input_dict['y_cells_widths'] = [0.25, 0.25, 0.25, 0.25]

In [45]:
input_dict['z_cells_widths'] = [0.25, 0.25, 0.25, 0.25]

The variables `x_cells_widths`, `y_cells_widths`, and `z_cells_widths` represent the widths of each cell in the x, y, and z directions, respectively. In order to ensure that adjacent cells fit together properly, each unique cell cannot have a unique size. Instead, the cell width specified applies to all cells in the plane perpendicular to the corresponding axis.

To clarify, if you assign a value of 1 to the first entry in the `x_cells_widths` array, it means that all cells with an x dimension of 1 will have a width of 1 unit.

Please note that the input arrays for `x_cells_widths`, `y_cells_widths`, and `z_cells_widths` should have dimensions of 1 by the number of nodes in the corresponding axis, and all entries in the array should be filled.

In [46]:
input_dict['x_boundry_conditions'] = [2, 2]

In [47]:
input_dict['y_boundry_conditions'] = [2, 2]

In [48]:
input_dict['z_boundry_conditions'] = [2, 2]

The variables `x_boundary_conditions`, `y_boundary_conditions`, and `z_boundary_conditions` represent the boundary conditions for each face of a cubic mesh. The specific entries are defined as follows:

`x_boundary_conditions`:

- 0: Boundary condition for the start face in the x-direction (xsbc).

- 1: Boundary condition for the end face in the x-direction (xebc).

`y_boundary_conditions`:

- 0: Boundary condition for the start face in the y-direction (ysbc).

- 1: Boundary condition for the end face in the y-direction (yebc).

`z_boundary_conditions`:

- 0: Boundary condition for the start face in the z-direction (zsbc).

- 1: Boundary condition for the end face in the z-direction (zebc).

The supported boundary conditions are as follows:

0 - Vacuum: Represents a vacuum boundary condition.

1 - Reflective: Indicates a reflective boundary condition.

2 - Fixed Inflow: Denotes a fixed inflow boundary condition.


In [49]:
input_dict['material_id'] = [ [ [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1] ], 
                              [ [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1] ],  
                              [ [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1] ],  
                              [ [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1] ] ]

The array `material_id` holds information about the materials used in a cubic mesh, which is intended for solving the neutron transport method. Please ensure that the dimensions of the array match the dimensions of the mesh cells, so that each spatial cell corresponds to a single material number. The cells should be ordered in the x, y, and z directions.

In [50]:
input_dict['quadrature_file'] = 'quad_file'

The `quad_file` refers to the quadrature file used in the neutron transport method. It is necessary only when the quadrature type is set to 2. In that case, providing a valid quadrature file is mandatory. If your quadrature type is different from 2, you can create an empty file and pass it as input for this parameter. Please refer to the formatting notes in the Spatial Solver Python API for further guidance.

In [51]:
input_dict['xs_file'] = 'xs'

The `xs_file` refers to the file that contains the cross-sectional data for the materials in your mesh, represented by the `material_id` array. The cross-sectional data should be formatted similar to the following example for a two-material xs file:
      
    ! Cross section file
    ! Material # 1
    ! Group #1
    5.894     ! Total XS
    1.8       ! Scattering matrix
    ! Material # 2
    ! Group #1
    1.237      ! Total XS
    0.12       ! Scattering matrix

In [52]:
input_dict['source_input_file'] = 'src_4.dat'

Note: See input file formatting notes in the Source File Formatting section.

In [53]:
input_dict['bc_input_file'] = 'bc_4.dat'

The `bc_input_file` is the file used to specify the boundary conditions for the neutron inflow on the faces of the mesh. Specifically, it is used for the boundary conditions that are designated as 2 (fixed inflow). To correctly format the boundary condition input file, please refer to the Boundry Condition formatting notes in the Spatial Solver Python API for detailed instructions and guidelines.

In [54]:
input_dict['flux_output_file'] = 'phi_4.ahot'

`flux_output_file` is the output file for the angular flux to be printed to.

In [55]:
input_dict['convergence_criterion'] = 1.e-12

The calculation is considered complete and the solution is considered converged when the flux in each cell at the current iteration is within a certain threshold called the `convergence_criterion` of the previous iteration. Typically, the convergence criterion is defined as the relative difference between the current and previous iterates. However, in situations where the flux values are very small, the absolute difference is used instead. For more details on the convergence tolerance and its determination, please refer to the Convergence Tolerance entry in the documentation or relevant resources.

In [56]:
input_dict['max_iterations'] = 6000

`max_iterations` is the maximum number of times the mesh should be sweeped.

**Note:** If this number of iterations is reached before the convergence criterion is satisfied, the calculation will terminate and report the current flux estimate.

In [57]:
input_dict['moments_converged'] = 0

`moments_converged` is the number of moments that should be converged upon for each quadrature in the solution space. Value for moments converged must be in range [0, `spatial_order_in`].

In [58]:
input_dict['converge_tolerence'] = 1.e-10

Converge tolerance is the tolerance that determines how the difference between flux iterates (df) that is used to determine convergence will be calculated. 

df is calculated as follows:

```python
   f = current flux value
   ct = convergence tolerance (value for this key, "converge_tolerance")
   f1 = flux value from the previous iteration
   If f1 > ct:
     df = absolute(f - f1) / f1
   Else
     df = absolute(f - f1)
```
The idea is to use the absolute difference instead of the relative difference between iterates when the flux is very small to help avoid rounding error.</pre>

In [59]:
dict_results = {}
dict_results = pyne.spatialsolver.solve(input_dict)

Before doing anything with the resulting data, you should check if the solver succesfully ran.
If the dictionary key 'success' is 1 (true), the job ran succesfully.  If it is 0 (false), you
were not so succesfull.

In [60]:
if(dict_results['success']):
    print('Yay, job ran succesfully!')
print(dict_results['success'])

Yay, job ran succesfully!
1


If you were not so lucky, and your job failed, the following key will give you the error message.  It will be 0 if the codes ran succesfully.

In [61]:
print(dict_results['error_msg'])

0


To get the results of the solver, create a output dictionary to store all the data from the solver, and then use solve to populate it!

In [62]:
print(dict_results['flux'])

[[[3.5265019965239373, 3.0926025752216213, 3.0926025752216213, 3.5265019965239373], [3.0926025752216204, 2.7320973265457127, 2.7320973265457127, 3.092602575221621], [3.092602575221621, 2.732097326545713, 2.732097326545713, 3.092602575221621], [3.526501996523937, 3.0926025752216213, 3.0926025752216213, 3.5265019965239364]], [[2.890218323639852, 2.612848109027689, 2.6128481090276896, 2.890218323639852], [2.6128481090276896, 2.385716782664984, 2.385716782664984, 2.6128481090276896], [2.6128481090276896, 2.385716782664984, 2.385716782664984, 2.61284810902769], [2.890218323639852, 2.61284810902769, 2.6128481090276896, 2.890218323639852]], [[2.8902183236398526, 2.61284810902769, 2.61284810902769, 2.8902183236398526], [2.6128481090276896, 2.3857167826649843, 2.3857167826649843, 2.6128481090276896], [2.6128481090276896, 2.3857167826649843, 2.3857167826649843, 2.6128481090276896], [2.8902183236398526, 2.61284810902769, 2.61284810902769, 2.8902183236398526]], [[3.526501996523937, 3.0926025752216

To obtain the total time taken by your job, you can retrieve it using the `total_time` key from the dictionary. This key provides the information about the elapsed time for the entire job execution.

In [63]:
print('Total solving time was: ')
print(dict_results['total_time'])

Total solving time was: 
0.004986000000000157


To obtain the absolute system time when the solver call finished, you can calculate it by adding the total job time (retrieved using the `total_time` key) and the solver start time (retrieved using the `time_start` key). The `time_start` key provides the system time when the solver call began, and by adding the total job time to the start time, you can determine the absolute system time when the solver call finished.

In [64]:
print('Solver call started at: ')
print(dict_results['time_start'])
print('Solver call finished at: ')
print(dict_results['time_start'] + dict_results['total_time'])

Solver call started at: 
2.292049
Solver call finished at: 
2.297035
